In [162]:
#Import Libraries

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [163]:
#Configuration Jobs

spark = SparkSession \
    .builder \
    .appName("Flights") \
    .getOrCreate()

In [164]:
#Load csv Data to DataFrame

In [165]:
#Flights
flightsDF= spark.read.option("header","true").csv("./Data/flights.csv")
#Airports
airportsDF= spark.read.option("header","true").csv("./Data/airports.csv")
#Airline
airlinesDF= spark.read.option("header","true").csv("./Data/airlines.csv")

In [166]:
#Functions to transform or remove columns
def RemoveColumnsFunction(df: DataFrame, columns_to_drop: ['']) -> DataFrame:
    return df.drop(*columns_to_drop)

In [167]:
#join airlines to flights to avoid code
flightsDF = flightsDF.join(airlinesDF,flightsDF.AIRLINE ==  airlinesDF.IATA_CODE,"inner").drop(flightsDF.AIRLINE).drop(col("IATA_CODE")).withColumnRenamed("AIRLINE","AIRLINE_") 
flightsDF = flightsDF.join(airportsDF,flightsDF.ORIGIN_AIRPORT ==  airportsDF.IATA_CODE,"inner").withColumnRenamed("CITY","CITY_DEPART_") 
columns_to_drop = ['ORIGIN_AIRPORT', 'IATA_CODE','AIRPORT','CITY','STATE','COUNTRY','LATITUDE','LONGITUDE']
flightsDF = RemoveColumnsFunction(flightsDF, columns_to_drop)
flightsDF = flightsDF.join(airportsDF,flightsDF.DESTINATION_AIRPORT ==  airportsDF.IATA_CODE,"inner").withColumnRenamed("CITY","CITY_ARRIVAL_") 
flightsDF = RemoveColumnsFunction(flightsDF, columns_to_drop)

In [168]:
# create date from year, month and day

In [169]:
cols=["YEAR","MONTH","DAY"]

In [170]:
flightsDF = flightsDF.withColumn("DATE",concat_ws("-",*cols).cast("date"))

In [171]:
flightsDF = flightsDF.drop(*cols)

In [172]:
# remove unecessary columns
columns_to_drop = [
  "TAXI_OUT",
  "TAXI_IN",
  "WHEELS_ON",
  "WHEELS_OFF",
  "YEAR",
  "MONTH",
  "DAY",
  "DAY_OF_WEEK",
  "AIR_SYSTEM_DELAY",
  "SECURITY_DELAY",
  "AIRLINE_DELAY",
  "LATE_AIRCRAFT_DELAY",
  "WEATHER_DELAY",
  "DIVERTED",
  "CANCELLED",
  "CANCELLATION_REASON",
  "FLIGHT_NUMBER",
  "TAIL_NUMBER",
  "AIR_TIME",
]

In [173]:
flightsDF = flightsDF.drop(*columns_to_drop)

In [174]:
flightsDF.show()

+-------------------+-------------------+--------------+---------------+--------------+------------+--------+-----------------+------------+-------------+--------------------+--------------+-----------------+----------+
|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|SCHEDULED_TIME|ELAPSED_TIME|DISTANCE|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|            AIRLINE_|  CITY_DEPART_|    CITY_ARRIVAL_|      DATE|
+-------------------+-------------------+--------------+---------------+--------------+------------+--------+-----------------+------------+-------------+--------------------+--------------+-----------------+----------+
|                SEA|               0005|          2354|            -11|           205|         194|    1448|             0430|        0408|          -22|Alaska Airlines Inc.|     Anchorage|          Seattle|2015-01-01|
|                PBI|               0010|          0002|             -8|           280|         279|    2330|           